In [1]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [2]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x[batch_idx], y[batch_idx]
        yield x_batch, y_batch

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [4]:
lr = 1e-2
epoch = 20
batch_size = 200
l2_regularization_constant = 1e-3 

In [5]:
n_inputs = 28 * 28  
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [6]:
from functools import partial

my_dense_layer = partial(
    tf.layers.dense, 
    activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l2_regularizer(1.)
)

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(x, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None, name="outputs")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [7]:
print(type(tf.GraphKeys.REGULARIZATION_LOSSES))
print(tf.GraphKeys.REGULARIZATION_LOSSES)

<class 'str'>
regularization_losses


In [8]:
print(type(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)))
print(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

<class 'list'>
[<tf.Tensor 'dnn/hidden1/kernel/Regularizer/l2_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'dnn/hidden2/kernel/Regularizer/l2_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'dnn/outputs/kernel/Regularizer/l2_regularizer:0' shape=() dtype=float32>]


In [9]:
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    entropy_loss = tf.reduce_mean(entropy, name="entropy_loss")
    l2_loss_list = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    l2_loss = tf.add_n(l2_loss_list)
    loss = tf.identity(entropy_loss + l2_regularization_constant * l2_loss, name="loss")

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    opt = tf.train.GradientDescentOptimizer(lr)
    train = opt.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
with tf.Session() as sess:
    init.run()
    for i in range(epoch):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train, feed_dict={x: x_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})
        print(i, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./save/my_model_final.ckpt")

0 Validation accuracy: 0.8282
1 Validation accuracy: 0.8776
2 Validation accuracy: 0.8934
3 Validation accuracy: 0.9042
4 Validation accuracy: 0.9124
5 Validation accuracy: 0.917
6 Validation accuracy: 0.9214
7 Validation accuracy: 0.924
8 Validation accuracy: 0.9276
9 Validation accuracy: 0.93
10 Validation accuracy: 0.9306
11 Validation accuracy: 0.9356
12 Validation accuracy: 0.9382
13 Validation accuracy: 0.9394
14 Validation accuracy: 0.9416
15 Validation accuracy: 0.9438
16 Validation accuracy: 0.9448
17 Validation accuracy: 0.9448
18 Validation accuracy: 0.9474
19 Validation accuracy: 0.9476
